### Setup

In [ ]:
import sys

if 'google.colab' in sys.modules:
    %pip install -q -U keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
%pip install 'git+https://github.com/jeslago/epftoolbox.git'

  Cloning https://github.com/jeslago/epftoolbox.git to /tmp/pip-req-build-z7rq9ixb
  Running command git clone --filter=blob:none --quiet https://github.com/jeslago/epftoolbox.git /tmp/pip-req-build-z7rq9ixb
  Resolved https://github.com/jeslago/epftoolbox.git to commit 47d6e0629f65ebd19d3c12cb5689dbad0c2ea078
  Preparing metadata (setup.py) ... done
  Created wheel for epftoolbox: filename=epftoolbox-1.0-py3-none-any.whl size=58746 sha256=ab7800c003a95a89d8668c42484f3b692d655b8e413b41f680ecc4cab6a23d24
  Stored in directory: /tmp/pip-ephem-wheel-cache-zljogs7s/wheels/f1/d7/c7/feb15c8f047f5b069656a52f3bcde89a7310a311f9cfac292a
Successfully built epftoolbox


In [ ]:
%pip install finta

In [ ]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 11.8 MB/s eta 0:00:00


In [ ]:
import os
import sys
from pathlib import Path

import finta
import keras_tuner as kt
import optuna
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import sklearn
import statsmodels
import tensorflow as tf
import tqdm
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

DRIVE_ROOT = Path('/content/drive/MyDrive')

PROJECT_ROOT = DRIVE_ROOT / 'projects/btc'
PROJECT_DATA_DIR = PROJECT_ROOT / 'data'
PROJECT_CLEANED_DIR = PROJECT_DATA_DIR / 'cleaned'
PROJECT_EXTERNAL_DIR = PROJECT_DATA_DIR / 'external'

CONFIG_FILENAME = 'config.yaml'
BTC_CLEANED_CSV_FILENAME = 'ohlcv_274_cleaned.csv'
FEDFUNDRATE_CSV_FILENAME = 'fedfunds.csv'
M2SL_CSV_FILENAME = 'm2sl.csv'

CONFIG_FILE_PATH = PROJECT_ROOT / CONFIG_FILENAME
BTC_CLEANED_FILE_PATH = PROJECT_CLEANED_DIR / BTC_CLEANED_CSV_FILENAME
FEDFUNDRATE_CSV_FILE_PATH = PROJECT_EXTERNAL_DIR / FEDFUNDRATE_CSV_FILENAME
M2SL_CSV_FILE_PATH = PROJECT_EXTERNAL_DIR / M2SL_CSV_FILENAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
project_root = os.path.abspath(PROJECT_ROOT)

if project_root not in sys.path:
  sys.path.append(project_root)

print(f'Project root \'{project_root}\' added to sys.path.')

Project root '/content/drive/MyDrive/projects/btc' added to sys.path.


In [ ]:
from src.analysis import *
from src.data_loader import *
from src.data_cleaner import *
from src.data_splitter import *
from src.model import *
from src.preprocessing import *
from src.visualization import *

In [ ]:
print('--- Python Interpreter and Standard Library Versions ---')
print(f'- python: {sys.version}')
print(f'- python info: {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}')

print(f'- os, hashlib, pathlib module: (version tied to Python interpreter)')

print('\n--- Third-Party Library Versions ---')
print(f'- keras_tuner: {kt.__version__}')
print(f'- optuna: {optuna.__version__}')
print(f'- matplotlib: {matplotlib.__version__}')
print(f'- numpy: {np.__version__}')
print(f'- pandas: {pd.__version__}')
print(f'- seaborn: {seaborn.__version__}')
print(f'- sklearn: {sklearn.__version__}')
print(f'- statsmodels: {statsmodels.__version__}')
print(f'- tensorflow: {tf.__version__}')
#print(f'- tqdm: {tqdm.__version__}')

print('\n--- Google Colab Specific Module Versions ---')
print(f'- google.colab.drive: (version tied to Colab environment)')

--- Python Interpreter and Standard Library Versions ---
- python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
- python info: 3.12.12
- os, hashlib, pathlib module: (version tied to Python interpreter)

--- Third-Party Library Versions ---
- keras_tuner: 1.4.7
- optuna: 4.5.0
- matplotlib: 3.10.0
- numpy: 2.0.2
- pandas: 2.2.2
- seaborn: 0.13.2
- sklearn: 1.6.1
- statsmodels: 0.14.5
- tensorflow: 2.19.0

--- Google Colab Specific Module Versions ---
- google.colab.drive: (version tied to Colab environment)


In [ ]:
%pip show epftoolbox

Name: epftoolbox
Version: 1.0
Summary: An open-access benchmark and toolbox for electricity price forecasting
Home-page: https://github.com/jeslago/epftoolbox
Author: Jesus Lago
Author-email: jesuslagogarcia@gmail.com
License: GNU AGPLv3
Location: /usr/local/lib/python3.12/dist-packages
Requires: hyperopt, keras, matplotlib, numpy, pandas, scikit-learn, scipy, statsmodels, tensorflow
Required-by: 


In [ ]:
%pip show finta

Name: finta
Version: 1.3
Summary: Common financial technical indicators implemented in Pandas.
Home-page: https://github.com/peerchemist/finta
Author: Peerchemist
Author-email: peerchemist@protonmail.ch
License: LGPLv3+
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, pandas
Required-by: 


### Load the cleaned dataset.

In [ ]:
cleaned_ds = load_btc_ds(BTC_CLEANED_FILE_PATH, 'date', True)


Loading dataset from: ohlcv_274_cleaned.csv...
Dataset loaded successfully.


### Perform linear interpolation to fill the missing values on March 15, 2025, to ensure a fair and correct comparison with other models.

In [ ]:
imputed_ds = fill_time_series_gaps(cleaned_ds, 'min')

### Resample the cleaned dataset into multiple timeframes.

In [ ]:
resampled_ds = resample_btc_data(imputed_ds)


--- Resampling BTC Data to Multiple Timeframes ---
- Resampling to hourly frequency...
- Resampling to daily frequency...
- Resampling to weekly frequency...
- Resampling to monthly frequency...
--- BTC Data Resampling Complete ---


### Split the resampled datasets into training, validation, and test sets of the target timeframe.

In [ ]:
train_ds, valid_ds, test_ds = split_btc_ds(resampled_ds['daily'], 'Day')


--- Splitting the BTC Dataset into Training, Validation, and Test Sets (Day) ---
--- BTC Dataset Splitting Complete ---


### Prepare the dataset by consolidating data splits,adding a comprehensive suite of features, and then splitting the data back.

In [ ]:
train_prep_ds, valid_prep_ds, test_prep_ds = prepare_feature_ds(
    train_ds, valid_ds, test_ds,
    ['open', 'high', 'low', 'close', 'volume'],
    FEDFUNDRATE_CSV_FILE_PATH, M2SL_CSV_FILE_PATH
)

= Feature Engineering and Dataset Preparation =
Step 1: Concatenating data splits...
- Combined train, validation, and test sets into a single DataFrame. Full shape: (4922, 5)

Step 2: Engineering new features...
- Calculated log returns for ['open', 'high', 'low', 'close', 'volume'].
- Calculated the high-low price range as a percentage of the open price.
- Calculated Simple Moving Average (SMA) for trend.
- Calculated Relative Strength Index (RSI) for momentum.
- Calculated Average True Range (ATR) for volatility.
- Calculated On-Balance Volume (OBV) for volume pressure.
- Calculated Moving Average Convergence Divergence (MACD).
- Engineered relationship features
- Calculated technical indicators.
- Macro data loaded and prepared.
- Macro data merged.
- Added macroeconomic features.

Step 3: Handling missing and infinite values...
- Replaced 0 infinite values (inf, -inf) with NaN.
- Forward-filled missing values to maintain data continuity.
- Shape before dropping NaNs: (4922, 28)
- 

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Naive Model Evaluation (1-Week)

In [ ]:
plot_btc_price_and_differencing(train_ds, 7, 'Day', 'btc_price_and_differencing_week.png')

Plot saved to 'outputs/plots/btc_price_and_differencing_week.png'.


In [ ]:
naive_metrics, naive_pred_out = train_and_forecast_with_naive_model(valid_prep_ds, 'close', 7, 'Validation', 'Days')

= Naive Model Training, Evaluation, and Forecasting on Validation Set (Horizon: 7 Days) =
--- Fitting Naive Model ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 4.7134%
- Directional Accuracy (DA): 44.7699%

--- Generating Final Naive Forecast (Threshold: 0.5%) ---
- Forecast for 2024-02-22: $52003.00
- Directional Signal for 2024-02-22: 1.0


In [ ]:
plot_residuals_analysis(naive_metrics, 'comp', 14, 'NAIVE', 'Week')
run_ljung_box_test(naive_metrics, 'comp', 14, 'NAIVE')
print(naive_metrics['comp'])

Plot saved to 'outputs/plots/naive_residuals_analysis_week'.
= Ljung-Box Test: NAIVE =
Lags tested: 14
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
               true     pred
2022-10-19  19226.0  19140.0
2022-10-20  19061.0  19375.0
2022-10-21  19189.0  19169.0
2022-10-22  19186.0  19105.0
2022-10-23  19507.0  19375.0
...             ...      ...
2024-02-11  48295.0  42779.0
2024-02-12  49745.0  42678.0
2024-02-13  49126.0  43176.0
2024-02-14  51475.0  43625.0
2024-02-15  52003.0  45283.0

[485 rows x 2 columns]


### Naive Model Evaluation (1-Month)

In [ ]:
plot_btc_price_and_differencing(train_ds, 30, 'Day', 'btc_price_and_differencing_month.png')

Plot saved to 'outputs/plots/btc_price_and_differencing_month.png'.


In [ ]:
naive_metrics, naive_pred_out = train_and_forecast_with_naive_model(valid_prep_ds, 'close', 30, 'Validation', 'Days')

= Naive Model Training, Evaluation, and Forecasting on Validation Set (Horizon: 30 Days) =
--- Fitting Naive Model ---

--- Evaluating Naive Model ---
- Mean Absolute Percentage Error (MAPE): 10.5813%
- Directional Accuracy (DA): 54.8611%

--- Generating Final Naive Forecast (Threshold: 0.5%) ---
- Forecast for 2024-03-16: $52003.00
- Directional Signal for 2024-03-16: 1.0


In [ ]:
plot_residuals_analysis(naive_metrics, 'comp', 90, 'NAIVE', 'Month')
run_ljung_box_test(naive_metrics, 'comp', 90, 'NAIVE')
print(naive_metrics['comp'])

Plot saved to 'outputs/plots/naive_residuals_analysis_month'.
= Ljung-Box Test: NAIVE =
Lags tested: 90
p-value: 0.0000
Result: Reject the null hypothesis (p < 0.05).
The residuals show evidence of autocorrelation.
               true     pred
2022-11-11  16806.0  19140.0
2022-11-12  16842.0  19375.0
2022-11-13  16499.0  19169.0
2022-11-14  16590.0  19105.0
2022-11-15  16721.0  19375.0
...             ...      ...
2024-02-11  48295.0  43424.0
2024-02-12  49745.0  42794.0
2024-02-13  49126.0  42882.0
2024-02-14  51475.0  42979.0
2024-02-15  52003.0  43005.0

[462 rows x 2 columns]
